# Apache Spark with Scala

More details and magic commands here: https://github.com/apache/incubator-toree/blob/master/etc/examples/notebooks/magic-tutorial.ipynb


In [15]:
import org.apache.spark._
import org.apache.spark.streaming._


object Analisador {

  // Args = path/to/text0.txt path/to/text1.txt
  def main(args: Array[String]) {

    // create Spark context with Spark configuration
    //não precisa, já que está em modo shell
    //val sc = new SparkContext(new SparkConf().setAppName("Contagem de Palavra"))
    //val ssc = new StreamingContext(sc, Seconds(1))
      
    println("TEXT1")

    // read first text file and split into lines
    val lines1 = sc.textFile(args(0))

    // contar palavras do texto 1 e imprimir as 5 palavras com as maiores ocorrencias (ordem DECRESCENTE)

    val words = lines1.flatMap(line => line.split(" "))

    val data = words.map(word => (word.replaceAll("[,.!?:;]",""), 1))
            .reduceByKey(_ + _)
            .filter(_._1.length > 3) // apenas palavras com mais de tres caracteres
            .map(_.swap) // esse trecho é necessário pois não há função sortByValue()
            .sortByKey(false,1)
            .map(_.swap)

    val output = data.take(5) // apenas as 5 mais frequentes

    output.foreach{ x => println(x._1 + "=" + x._2.toString)} // imprimir em cada linha: "palavra=numero"

    println("TEXT2")

    // read second text file and split each document into words
    val lines2 = sc.textFile(args(1))

    // transformações do texto 2 análogas ao texto 1

    val words2 = lines2.flatMap(line => line.split(" "))

    val data2 = words2.map(word => (word.replaceAll("[,.!?:;]",""),1))
            .reduceByKey(_ + _)
            .filter(_._1.length > 3)
            .map(_.swap)
            .sortByKey(false,1)
            .map(_.swap)

    val output2 = data2.take(5)

    output2.foreach{ x => println(x._1 + "=" + x._2.toString)}

    // comparar resultado e imprimir na ordem ALFABETICA todas as palavras que aparecem MAIS que 100 vezes nos 2 textos

    println("COMMON")

    val filt1 = data.filter(_._2 > 100).sortByKey().keys // palavras do texto 1

    val filt2 = data2.filter(_._2 > 100).sortByKey().keys // palavras do texto 2

    val data3 =  filt1.intersection(filt2) // intersecção

    val output3 = data3.map(word => (word,1)).sortByKey() // rdd expandido pois não consegui ordenar um rdd com array de apenas uma string

    output3.foreach{ x => println(x._1)} // imprimir em cada linha: "palavra"

  }

}





Running ...

In [18]:
val files = Array("./resources/data/input1.txt", "./resources/data/input2.txt")

Analisador.main(files)

TEXT1
said=451
Alice=377
that=224
with=170
very=126
TEXT2
that=733
with=435
were=363
from=312
they=225
COMMON


Desired output:
TEXT1
said=456
alice=377
that=234
with=172
very=139
TEXT2
that=759
with=448
were=365
from=326
they=302
COMMON
little
said
that
they
this
with
